# Preprocessing the weather data
1) Download the weather data from climate.weather.gc.ca  
2) Mimick the naming convention of the files and make them as array  
3) Find missing data  
4) Iterate the array to open files in time order and Create dataset  
5) Fill the missing data in dataset by Interpolation  

### 1. Download the data files
Source : http://climate.weather.gc.ca  
Desc: Hourly Weather Data from Jan. 2015 to July. 2018 in Kingston  
Station : Kingston climate, Ontario, Canada  
Station ID: 47267  


**Linux Command to get weather data**

for year in `seq 2015 2018`;do for month in `seq 1 12`;do wget --content-disposition "http://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=47267&Year=${year}&Month=${month}&Day=14&timeframe=1&submit= Download+Data" ;done;done
  
for year in `seq 2019 2019`;do for month in `seq 1 7`;do wget --content-disposition "http://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=47267&Year=${year}&Month=${month}&Day=14&timeframe=1&submit= Download+Data" ;done;done

**To see the detail of the query, please refer below**

https://drive.google.com/drive/folders/1WJCDEU34c60IfOnG4rv5EPZ4IhhW9vZH  

http://climate.weather.gc.ca/climate_data/hourly_data_e.html?hlyRange=2008-07-15%7C2019-08-05&dlyRange=2008-07-15%7C2019-08-05&mlyRange=%7C&StationID=47267&Prov=ON&urlExtension=_e.html&searchType=stnProv&optLimit=yearRange&StartYear=2019&EndYear=2019&selRowPerPage=25&Line=77&Month=8&Day=5&lstProvince=ON&timeframe=1&Year=2019


### 2. Set up the array of file names

Using the naming convention, set up the file name array  

e.g.) Jan. 2015 data file's name  =  "eng-hourly-01012015-01312015.csv"  
which means data between 01-01-2015 and 01-31-2015


In [264]:
import calendar

years = [2015,2016,2017,2018]
filenames = []

for year in years:
    for month in range(1,13):
        filename = 'eng-hourly-{month:02d}01{year}-{month:02d}{last:02d}{year}.csv'.format(month=month, year=year, last=calendar.monthrange(year, month)[1])
        filenames.append(filename)

years = [2019]
for year in years:
    for month in range(1, 8):
        filename = 'eng-hourly-{month:02d}01{year}-{month:02d}{last:02d}{year}.csv'.format(month=month, year=year, last=calendar.monthrange(year, month)[1])
        filenames.append(filename)
        
print(filenames)



['eng-hourly-01012015-01312015.csv', 'eng-hourly-02012015-02282015.csv', 'eng-hourly-03012015-03312015.csv', 'eng-hourly-04012015-04302015.csv', 'eng-hourly-05012015-05312015.csv', 'eng-hourly-06012015-06302015.csv', 'eng-hourly-07012015-07312015.csv', 'eng-hourly-08012015-08312015.csv', 'eng-hourly-09012015-09302015.csv', 'eng-hourly-10012015-10312015.csv', 'eng-hourly-11012015-11302015.csv', 'eng-hourly-12012015-12312015.csv', 'eng-hourly-01012016-01312016.csv', 'eng-hourly-02012016-02292016.csv', 'eng-hourly-03012016-03312016.csv', 'eng-hourly-04012016-04302016.csv', 'eng-hourly-05012016-05312016.csv', 'eng-hourly-06012016-06302016.csv', 'eng-hourly-07012016-07312016.csv', 'eng-hourly-08012016-08312016.csv', 'eng-hourly-09012016-09302016.csv', 'eng-hourly-10012016-10312016.csv', 'eng-hourly-11012016-11302016.csv', 'eng-hourly-12012016-12312016.csv', 'eng-hourly-01012017-01312017.csv', 'eng-hourly-02012017-02282017.csv', 'eng-hourly-03012017-03312017.csv', 'eng-hourly-04012017-043020

### 3. Find missing data

In [265]:
def find_missing_data(filenames):
    
    
    raw_file = open(filenames[0], "r")
    raw_data = raw_file.readlines()[16:]
    raw_file.close()
    
    for index in range(1, len(filenames)):
        raw_file = open(filenames[index], "r")
        raw_data = raw_data + raw_file.readlines()[16:]
        raw_file.close()

    new_filename = "missing_data.csv"
    f = open(new_filename, "w")
    output = []
    for item in raw_data:
        if item.count(",") is not 23: # if full data, it is supposed to have 23 of , (seperator)
            output.append(item.replace("\"", ""))
    f.writelines(output)
    f.close()
                          
find_missing_data(filenames)

# some large chunk of missing data were found. 
# e.g.) 2017-11-26 ~ 2017-12-06
# e.g.) 2018-04-23 ~ 2018-04-27

missing_data_file = open("missing_data.csv", "r")
missing_data = missing_data_file.readlines()
missing_data_file.close()
for item in missing_data[190:195]:
    print(item)
    
print(len(missing_data)) # total 550 days have missing information

2017-11-26 06:00,2017,11,26,06:00

2017-11-26 07:00,2017,11,26,07:00

2017-11-26 08:00,2017,11,26,08:00

2017-11-26 09:00,2017,11,26,09:00

2017-11-26 10:00,2017,11,26,10:00

550


### 4. Create Dataset (Pandas Dataframe)

In [266]:
from pandas import read_csv
import pandas as pd

dataset = read_csv(filenames[0], skiprows = 15, header = 0)
for count in range(1, len(filenames)):
    dataset = pd.concat([dataset, read_csv(filenames[count], skiprows = 15, header = 0)])

print(dataset.shape)

(40152, 24)


In [267]:
# unnecessary columns in raw data
to_be_deleted = [1, 2, 3, 4, 6, 8, 10, 12, 14, 15, 16, 18, 19, 20, 21, 22, 23] 
dataset = dataset.drop(dataset.columns[to_be_deleted], axis=1)

In [268]:
# set Date/Time colum to index

# https://appdividend.com/2019/01/26/pandas-set-index-example-python-set_index-tutorial/
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html

dataset.set_index(dataset.columns[0], inplace=True)

# missing data
print(dataset[380:395]) # missing data at row 62

                  Temp (°C)  Dew Point Temp (°C)  Rel Hum (%)  \
Date/Time                                                       
2015-01-16 20:00      -14.1                -22.2         50.0   
2015-01-16 21:00      -15.8                -22.9         54.0   
2015-01-16 22:00        NaN                  NaN          NaN   
2015-01-16 23:00        NaN                  NaN          NaN   
2015-01-17 00:00        NaN                  NaN          NaN   
2015-01-17 01:00        NaN                  NaN          NaN   
2015-01-17 02:00        NaN                  NaN          NaN   
2015-01-17 03:00        NaN                  NaN          NaN   
2015-01-17 04:00        NaN                  NaN          NaN   
2015-01-17 05:00        NaN                  NaN          NaN   
2015-01-17 06:00        NaN                  NaN          NaN   
2015-01-17 07:00        NaN                  NaN          NaN   
2015-01-17 08:00        NaN                  NaN          NaN   
2015-01-17 09:00        N

In [269]:
# Check how many missing data are there
dataset.isnull().sum() 

Temp (°C)              560
Dew Point Temp (°C)    551
Rel Hum (%)            550
Wind Dir (10s deg)     770
Wind Spd (km/h)        550
Stn Press (kPa)        551
dtype: int64

### 5. Fill the missing data

I'v considered which one to use, Fillna() or Interpolate()  
But for this time, I chose Interpolate() since the data is time series

**Pandas - fillna method**

https://jakevdp.github.io/PythonDataScienceHandbook/03.04-missing-values.html

In [ ]:
# dataset.fillna(dataset.mean(), inplace=True)
# dataset.fillna(method='ffill', inplace = True)
# dataset.fillna(method='bfill', inplace = True)

**Pandas - Interpolation**

https://www.geeksforgeeks.org/python-pandas-dataframe-interpolate/  
https://stackoverflow.com/questions/34934511/cannot-interpolate-dataframe-even-if-most-of-the-data-is-filled  
https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html#interpolation  


In [270]:
# Interpolation
dataset.interpolate(method='linear', limit_direction ='both', limit=500, inplace=True)
print(dataset[380:395])

                  Temp (°C)  Dew Point Temp (°C)  Rel Hum (%)  \
Date/Time                                                       
2015-01-16 20:00 -14.100000           -22.200000    50.000000   
2015-01-16 21:00 -15.800000           -22.900000    54.000000   
2015-01-16 22:00 -15.969231           -22.892308    54.923077   
2015-01-16 23:00 -16.138462           -22.884615    55.846154   
2015-01-17 00:00 -16.307692           -22.876923    56.769231   
2015-01-17 01:00 -16.476923           -22.869231    57.692308   
2015-01-17 02:00 -16.646154           -22.861538    58.615385   
2015-01-17 03:00 -16.815385           -22.853846    59.538462   
2015-01-17 04:00 -16.984615           -22.846154    60.461538   
2015-01-17 05:00 -17.153846           -22.838462    61.384615   
2015-01-17 06:00 -17.323077           -22.830769    62.307692   
2015-01-17 07:00 -17.492308           -22.823077    63.230769   
2015-01-17 08:00 -17.661538           -22.815385    64.153846   
2015-01-17 09:00 -17.8307

In [271]:
dataset.isnull().sum() # No N/A now

Temp (°C)              0
Dew Point Temp (°C)    0
Rel Hum (%)            0
Wind Dir (10s deg)     0
Wind Spd (km/h)        0
Stn Press (kPa)        0
dtype: int64

### Save the processed data to .csv file

In [273]:
dataset.to_csv (r'export_data.csv', header=True) #Don't forget to add '.csv' at the end of the path

In [276]:
# Make suer it's done properly
dataset = read_csv('export_data.csv', header = 0)
dataset.set_index(dataset.columns[0], inplace=True)

In [279]:
print(dataset[:5])
print(dataset[-5:])

                  Temp (°C)  Dew Point Temp (°C)  Rel Hum (%)  \
Date/Time                                                       
2015-01-01 00:00       -5.9                -11.8         63.0   
2015-01-01 01:00       -5.9                -13.6         54.0   
2015-01-01 02:00       -6.1                -14.2         53.0   
2015-01-01 03:00       -5.8                -13.2         56.0   
2015-01-01 04:00       -5.7                -11.9         62.0   

                  Wind Dir (10s deg)  Wind Spd (km/h)  Stn Press (kPa)  
Date/Time                                                               
2015-01-01 00:00                26.0             28.0           100.21  
2015-01-01 01:00                26.0             21.0           100.19  
2015-01-01 02:00                26.0             27.0           100.17  
2015-01-01 03:00                25.0             28.0           100.14  
2015-01-01 04:00                25.0             24.0           100.14  
                  Temp (°C)  Dew 